<a href="https://colab.research.google.com/github/martinpius/keras_Functional_API_architecture/blob/main/Alien_Revival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount("/content/drive/", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are using Google colab with tensorflow version: {tf.__version__}")
except Exception as e:
  COLAB = False
  print(f"{type(e)}: {e}\n...Please Load Your Drive...")

def time_fmt(t):
  h = int(t / (60 * 60))
  m = int(t % (60 * 60)/60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"

Mounted at /content/drive/
You are using Google colab with tensorflow version: 2.4.0


In [29]:
import tensorflow as tf
import numpy as np
import time
time_fmt(123.4903)

'0: 02: 03.00'

In [30]:
#End to end autoencoder's for cifar10 dataset
#We will train the autoencoder in two different way
#As a ussual MLP and as a CNN

In [31]:
#The ussual MLP 

In [32]:
#Defining the sampling class (layer subclassing): We sample from the normal distribution to instantiate the decoder's inputs

In [33]:
class SampleGenerator(tf.keras.layers.Layer):
  def call(self, inputs):
    mu, sigma = inputs
    dim1 = tf.shape(mu)[0]
    dim2 = tf.shape(mu)[1]
    eps = tf.keras.backend.random_normal(shape = (dim1, dim2))
    return mu + tf.exp(0.5 * sigma) * eps
  

In [34]:
#Defining  the encoder's class (layer-subclassing) which encode the original data/image to some representation

In [35]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, hidden = 64, intermediate = 128, name = 'encoder', **kwargs):
    super(Encoder, self).__init__(name = name, **kwargs)
    self.layer1 = tf.keras.layers.Dense(units = hidden, activation = 'relu', kernel_initializer = 'random_normal')
    self.layer2 = tf.keras.layers.Dense(units = intermediate, activation = 'relu', kernel_initializer = 'random_normal')
    self.mu = tf.keras.layers.Dense(units = intermediate, activation = 'relu', kernel_initializer = 'random_normal')
    self.sigma = tf.keras.layers.Dense(units = intermediate, activation = 'relu', kernel_initializer = 'random_normal')
    self.sample_generator = SampleGenerator()
  
  def call(self, inputs):
    x = self.layer1(inputs)
    x = self.layer2(x)
    mu = self.mu(x)
    sigma = self.sigma(x)
    z = self.sample_generator((mu, sigma))
    return mu, sigma, z


In [36]:
#Defining a decoder's block. This is a layer-subclassing procedure to recreate the original data/image

In [37]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self,original_dim, hidden = 64,intermediate = 128, name = 'decoder', **kwargs):
    super(Decoder, self).__init__(name = name, **kwargs)
    self.dense1 = tf.keras.layers.Dense(units = hidden, activation = 'relu', kernel_initializer = 'random_normal')
    self.dense2 = tf.keras.layers.Dense(units = intermediate, activation = 'relu', kernel_initializer='random_normal')
    self.out = tf.keras.layers.Dense(units = original_dim, activation ='sigmoid')
  
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    return self.out(x)

In [38]:
#The autoencoder's class: (this is a model-subclassing) procedure. Here we combine encoder's and the decoder's to get an end-to-end model

In [39]:
class MyAutoEncoder(tf.keras.Model):
  def __init__(self, original_dim, hidden = 64, intermediate = 128, name = 'vae', **kwargs):
    super(MyAutoEncoder, self).__init__(name = name, **kwargs)
    self.original_dim = original_dim
    self.encoder = Encoder(hidden = hidden, intermediate = intermediate)
    self.decoder = Decoder(original_dim = original_dim, hidden = hidden)
  
  def call(self, inputs):
    mu, sigma, z = self.encoder(inputs)
    re_build = self.decoder(z)
    kl_Div = -0.5 * tf.reduce_mean(sigma - tf.square(mu) - tf.exp(sigma) + 1)
    self.add_loss(kl_Div)
    return re_build


In [40]:
#Instantiate our model 

In [41]:
original_dim = 784 #This is a flat version of  32,32,3 cifar10 image

In [42]:
VAE = MyAutoEncoder(original_dim,64,128)

In [43]:
#Get the data and pre-process (We only need a training set)

In [44]:
(x_train, _),(_, _) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [45]:
x_train.shape

(60000, 28, 28)

In [46]:
x_train = x_train.reshape(60000,784).astype('float32')/255.0

In [47]:
#Change to tensorflow dataset for easy streaming
x_train.shape
train_dfm = tf.data.Dataset.from_tensor_slices(x_train)

In [48]:
train_dfm = train_dfm.shuffle(buffer_size = 1024).batch(64)

In [49]:
epochs = 10

In [50]:
main_loss = tf.keras.losses.MeanAbsoluteError()
metric_fn = tf.keras.metrics.Mean()
optimizer = tf.keras.optimizers.RMSprop(learning_rate = 1e-3)

In [51]:
#The training loop

In [52]:
tic = time.time()
for epoch in range(epochs):
  print(f"Here we begin training at epoch: {epoch}")

  for step, x_batch_train in enumerate(train_dfm):
    with tf.GradientTape() as tape:
      re_build = VAE(x_batch_train)
      loss = main_loss(x_batch_train, re_build)
      loss+=sum(VAE.losses)#Adding the KL_Divergence loss for autoencoder
    grads = tape.gradient(loss, VAE.trainable_weights)
    optimizer.apply_gradients(zip(grads, VAE.trainable_weights))
    metric_fn(loss)
    if step % 100 == 0:
      print("step %d: mean loss = %.4f" % (step, metric_fn.result()))
toc = time.time()
print(f"Time elapsed: {time_fmt(tic - toc)}")

Here we begin training at epoch: 0
step 0: mean loss = 0.4745
step 100: mean loss = 0.1680
step 200: mean loss = 0.1469
step 300: mean loss = 0.1395
step 400: mean loss = 0.1362
step 500: mean loss = 0.1337
step 600: mean loss = 0.1323
step 700: mean loss = 0.1311
step 800: mean loss = 0.1303
step 900: mean loss = 0.1295
Here we begin training at epoch: 1
step 0: mean loss = 0.1293
step 100: mean loss = 0.1289
step 200: mean loss = 0.1287
step 300: mean loss = 0.1283
step 400: mean loss = 0.1282
step 500: mean loss = 0.1278
step 600: mean loss = 0.1277
step 700: mean loss = 0.1274
step 800: mean loss = 0.1273
step 900: mean loss = 0.1271
Here we begin training at epoch: 2
step 0: mean loss = 0.1270
step 100: mean loss = 0.1269
step 200: mean loss = 0.1269
step 300: mean loss = 0.1267
step 400: mean loss = 0.1267
step 500: mean loss = 0.1266
step 600: mean loss = 0.1265
step 700: mean loss = 0.1264
step 800: mean loss = 0.1264
step 900: mean loss = 0.1262
Here we begin training at epoch

In [53]:
#Autoencoder's for cifar10 data

In [54]:
inputs = tf.keras.Input(shape = (32,32,3), name = 'cifar10_img')
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'valid')(inputs)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'valid')(x)
x = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', padding = 'valid')(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'valid')(x)
x = tf.keras.layers.BatchNormalization()(x)
out = tf.keras.layers.GlobalMaxPooling2D()(x)

x = tf.keras.layers.Reshape((8,8,1))(out)
x = tf.keras.layers.Conv2DTranspose(filters = 64, kernel_size = (3,3), activation = 'relu')(x)
x = tf.keras.layers.Conv2DTranspose(filters = 128, kernel_size = (3,3), activation = 'relu')(x)
x = tf.keras.layers.UpSampling2D(size = (2,2))(x)
x = tf.keras.layers.Conv2DTranspose(filters = 64, kernel_size = (3,3), activation = 'relu')(x)
x = tf.keras.layers.Conv2DTranspose(filters = 128, kernel_size = (3,3), activation = 'relu')(x)
x = tf.keras.layers.Conv2DTranspose(filters = 128, kernel_size = (3,3), activation = 'relu')(x)
og_img = tf.keras.layers.Conv2DTranspose(filters = 3, kernel_size = (3,3), activation = 'relu', name = 'org_img')(x)
vae = tf.keras.Model(inputs = inputs, outputs = og_img)

In [55]:
vae.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cifar10_img (InputLayer)     [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 28, 28, 64)        73792     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 10, 10, 64)        73792     
_________________________________________________________________
batch_normalization_10 (Batc (None, 10, 10, 64)        256 

In [56]:
my_loss = tf.keras.losses.MeanAbsoluteError()
my_metric = tf.keras.metrics.Mean()
my_optimizer = tf.keras.optimizers.RMSprop(learning_rate = 1e-3)


In [57]:
epochs = 15

In [58]:
#Load cifar10 dataset
(x_train, _),(_, _) = tf.keras.datasets.cifar10.load_data()

In [59]:
#Preprocess the data and convert to tensorflow datatype in batches of size 64 each
x_train = x_train.astype('float32')/255.0

In [60]:
x_train.shape

(50000, 32, 32, 3)

In [61]:
train_dfm = tf.data.Dataset.from_tensor_slices(x_train)
train_dfm = train_dfm.shuffle(buffer_size = 1024).batch(64)

In [62]:
tic = time.time()
for epoch in range(epochs):
  print(f"The start of epoch: {epoch}")
  for step,x_train_batch in enumerate(train_dfm):
    with tf.GradientTape() as tape:
      org_img = vae(x_train_batch)
      loss = my_loss(x_train_batch, org_img)
      loss+= loss
    grads = tape.gradient(loss, vae.trainable_weights)
    my_optimizer.apply_gradients(zip(grads, vae.trainable_weights))
    my_metric(loss)
    if step % 100 == 0:
      print("step %d: mean loss = %.4f" % (step, my_metric.result()))
toc = time.time()
print(f"time elapsed: {tic - toc}")

The start of epoch: 0
step 0: mean loss = 0.9166
step 100: mean loss = 0.5413
step 200: mean loss = 0.4840
step 300: mean loss = 0.4627
step 400: mean loss = 0.4515
step 500: mean loss = 0.4424
step 600: mean loss = 0.4344
step 700: mean loss = 0.4257
The start of epoch: 1
step 0: mean loss = 0.4190
step 100: mean loss = 0.4120
step 200: mean loss = 0.4059
step 300: mean loss = 0.4003
step 400: mean loss = 0.3949
step 500: mean loss = 0.3904
step 600: mean loss = 0.3860
step 700: mean loss = 0.3821
The start of epoch: 2
step 0: mean loss = 0.3789
step 100: mean loss = 0.3755
step 200: mean loss = 0.3723
step 300: mean loss = 0.3694
step 400: mean loss = 0.3666
step 500: mean loss = 0.3641
step 600: mean loss = 0.3616
step 700: mean loss = 0.3593
The start of epoch: 3
step 0: mean loss = 0.3574
step 100: mean loss = 0.3552
step 200: mean loss = 0.3531
step 300: mean loss = 0.3513
step 400: mean loss = 0.3494
step 500: mean loss = 0.3477
step 600: mean loss = 0.3459
step 700: mean loss =

In [64]:
print(f"time elapsed: {time_fmt(toc - tic)}")

time elapsed: 0: 07: 32.00
